In [4]:
pip install bs4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install html5lib

     ---------------------------------------- 0.0/112.2 kB ? eta -:--:--
     --- ------------------------------------ 10.2/112.2 kB ? eta -:--:--
     -------------------------------- ------ 92.2/112.2 kB 1.1 MB/s eta 0:00:01
     -------------------------------------- 112.2/112.2 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
from selenium import webdriver 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

In [54]:
URL = "https://www.transfermarkt.co.uk/premier-league/marktwerte/wettbewerb/GB1"

In [60]:

headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
r = requests.get(url=URL, headers=headers)
soup = BeautifulSoup(r.content, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <script type="text/javascript">
   !function () { var e = function () { var e, t = "__tcfapiLocator", a = [], n = window; for (; n;) { try { if (n.frames[t]) { e = n; break } } catch (e) { } if (n === window.top) break; n = n.parent } e || (!function e() { var a = n.document, r = !!n.frames[t]; if (!r) if (a.body) { var i = a.createElement("iframe"); i.style.cssText = "display:none", i.name = t, a.body.appendChild(i) } else setTimeout(e, 5); return !r }(), n.__tcfapi = function () { for (var e, t = arguments.length, n = new Array(t), r = 0; r < t; r++)n[r] = arguments[r]; if (!n.length) return a; if ("setGdprApplies" === n[0]) n.length > 3 && 2 === parseInt(n[1], 10) && "boolean" == typeof n[3] && (e = n[3], "function" == typeof n[2] && n[2]("set", !0)); else if ("ping" === n[0]) { var i = { gdprApplies: e, cmpLoaded: !1, cmpStatus: "stub" }; "function" == typeof n[2] && n[2](i) } else a.push(n) }, n.addEventListener("message", (function (e) {

In [153]:
players_list = []
age = []
position = []
value = []
nationality = []

In [144]:
players = soup.find_all("td", class_="hauptlink")
    

In [154]:
players

[<td class="hauptlink">
 <a href="/erling-haaland/profil/spieler/418560" title="Erling Haaland">Erling Haaland</a> </td>,
 <td class="rechts hauptlink"><a href="/erling-haaland/marktwertverlauf/spieler/418560">€170.00m</a> </td>,
 <td class="hauptlink">
 <a href="/phil-foden/profil/spieler/406635" title="Phil Foden">Phil Foden</a> </td>,
 <td class="rechts hauptlink"><a href="/phil-foden/marktwertverlauf/spieler/406635">€110.00m</a> </td>,
 <td class="hauptlink">
 <a href="/bukayo-saka/profil/spieler/433177" title="Bukayo Saka">Bukayo Saka</a> </td>,
 <td class="rechts hauptlink"><a href="/bukayo-saka/marktwertverlauf/spieler/433177">€110.00m</a> <span class="icons_sprite green-arrow-ten" title="Previous market value: €100.00m"> </span></td>,
 <td class="hauptlink">
 <a href="/harry-kane/profil/spieler/132098" title="Harry Kane">Harry Kane</a> </td>,
 <td class="rechts hauptlink"><a href="/harry-kane/marktwertverlauf/spieler/132098">€90.00m</a> </td>,
 <td class="hauptlink">
 <a href="

In [155]:
for player in players:
    players_list.append(player.text)

In [162]:
players_list

['\nErling Haaland ',
 '€170.00m\xa0',
 '\nPhil Foden ',
 '€110.00m\xa0',
 '\nBukayo Saka ',
 '€110.00m\xa0\xa0',
 '\nHarry Kane ',
 '€90.00m\xa0',
 '\nEnzo Fernández ',
 '€85.00m\xa0\xa0',
 '\nDeclan Rice ',
 '€80.00m\xa0',
 '\nRodri ',
 '€80.00m\xa0',
 '\nMartin Ødegaard ',
 '€80.00m\xa0\xa0',
 '\nKevin De Bruyne ',
 '€80.00m\xa0',
 '\nBernardo Silva ',
 '€80.00m\xa0',
 '\nMarcus Rashford ',
 '€80.00m\xa0\xa0',
 '\nGabriel Jesus ',
 '€75.00m\xa0',
 '\nLuis Díaz ',
 '€75.00m\xa0',
 '\nRúben Dias ',
 '€75.00m\xa0',
 '\nBruno Fernandes ',
 '€75.00m\xa0',
 '\nMohamed Salah ',
 '€70.00m\xa0\xa0',
 '\nReece James ',
 '€70.00m\xa0',
 '\nJack Grealish ',
 '€70.00m\xa0',
 '\nDarwin Núñez ',
 '€70.00m\xa0',
 '\nGabriel Martinelli ',
 '€70.00m\xa0\xa0',
 '\nAntony ',
 '€70.00m\xa0\xa0',
 '\nWesley Fofana ',
 '€65.00m\xa0',
 '\nMason Mount ',
 '€65.00m\xa0\xa0',
 '\nTrent Alexander-Arnold ',
 '€65.00m\xa0\xa0',
 '\nCody Gakpo ',
 '€60.00m\xa0',
 'Bukayo Saka',
 '€110.00m\xa0\xa0',
 'Enzo Fernánd